# Estimate attractor dimension given N dimensional time-series data

In [1]:
import skdim
import numpy as np



In [17]:
#generate data : np.array (n_points x n_dim). Here a uniformly sampled 5-ball embedded in 10 dimensions
data = np.zeros((10000,10))
data[:,:3] = skdim.datasets.hyperBall(n = 10000, d = 3, radius = 1, random_state = 0)

#estimate global intrinsic dimension
danco = skdim.id.DANCo().fit(data)
#estimate local intrinsic dimension (dimension in k-nearest-neighborhoods around each point):
lpca = skdim.id.lPCA().fit_pw(data,
                              n_neighbors = 100,
                              n_jobs = 1)
                            
#get estimated intrinsic dimension
print(danco.dimension_, np.mean(lpca.dimension_pw_)) 

2.9330115407758512 3.0


In [18]:
# Correlation dimension

corr_dim = skdim.id.CorrInt().fit(data)
corr_dim.dimension_

2.9330291542031164

3.6068171672424416

In [ ]:
# Load an example 